In [26]:
from transformers import DistilBertTokenizerFast, DistilBertModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import torch.nn.functional as F
import numpy as np
import pandas as pd
from math import ceil
from numpy import load
import pandas as pd

In [2]:
# Setup backend device
if torch.cuda.is_available():
    # Check for CUDA (traditional GPUs)
    device = torch.device("cuda")
    print("PyTorch is using CUDA.")
elif torch.backends.mps.is_available():
    # Check for MPS (Apple Silicon GPUs)
    device = torch.device("mps")
    print("PyTorch is using MPS.")
else:
    device = torch.device("cpu")
    print("PyTorch is using CPU.")

def load_emb_data(file_name):
    # load dict of arrays
    dict_data = load(f"../../data/processed/{file_name}.npz")
    # extract the first array
    return dict_data['arr_0']

PyTorch is using CUDA.


In [27]:
# Read index
title_index_error = load_emb_data("title_index_error")
title_index = load_emb_data("title_index")
title_label = load_emb_data("title_label")

feature_index_error = load_emb_data("feature_index_error")
feature_index = load_emb_data("feature_index")
feature_label = load_emb_data("feature_label")

In [20]:
# Join error index
joined_error_index = np.unique(np.hstack([title_index_error, feature_index_error]))

df_title = pd.DataFrame({"original_index": title_index, "label": title_label})
df_feature = pd.DataFrame({"original_index": feature_index, "label": feature_label})
df = df_title.merge(df_feature, how="inner")

In [38]:
df_join_correct = df[~np.isin(df.original_index, joined_error_index)]
df_join_correct.label.value_counts()

In [54]:
df_join_error = df[np.isin(df.original_index, joined_error_index)]
df_join_error.label.value_counts()

label
2    36054
0    30132
1     5090
Name: count, dtype: int64

In [96]:
np.random.seed(42)
num_error_select = 5000
# Randomly select 5000 data for each category in the error index set
selected_id = np.array([])
for i in range(3):
    temp = df_join_error.original_index[df_join_error.label == i].sample(n=num_error_select).to_numpy()
    selected_id = np.hstack([selected_id, temp])

num_correct_select = 7500
# Randomly select 7500 data for each category in the error index set
for i in range(3):
    temp = df_join_correct.original_index[df_join_correct.label == i].sample(n=num_correct_select).to_numpy()
    selected_id = np.hstack([selected_id, temp])

selected_id = selected_id.astype(int)

df_select = df[df.original_index.isin(selected_id)]

np.savez_compressed(f'../../data/processed/combine_index.npz', df_select.original_index.to_numpy())
np.savez_compressed(f'../../data/processed/combine_label.npz', df_select.label.to_numpy())

In [99]:
title_bert_vec = load_emb_data("title_bert_vec")
selected_vec = title_bert_vec[np.isin(title_index, df_select.original_index)]
np.savez_compressed(f'../../data/processed/combine_title.npz', selected_vec)

In [104]:
feature_bert_vec = load_emb_data("feature_bert_vec")
selected_vec = feature_bert_vec[np.isin(feature_index, df_select.original_index)]
np.savez_compressed(f'../../data/processed/combine_feature.npz', selected_vec)